In [1]:
import pandas as pd
import os
import pathlib
import time

## all tickers

In [2]:
df1 = pd.read_csv('./open/NASDAQ_DT_FC_STK_QUT.csv')
ticker1 = df1['tck_iem_cd'].unique().tolist()
len(ticker1)

2743

In [3]:
df2 = pd.read_csv('./open/NASDAQ_FC_STK_IEM_IFO.csv',encoding='cp949')
ticker2 = df2['tck_iem_cd'].unique().tolist()
len(ticker2)

2743

In [4]:
import pathlib

rss_ifo = list(pathlib.Path('./open/NASDAQ_RSS_IFO/').glob('*.csv'))

ticker3 = []
for rss in rss_ifo:
    temp = pd.read_csv(rss,encoding='cp949')
    ticker = temp['tck_iem_cd'].unique().tolist()
    ticker3.extend(ticker)
    
ticker3 = list(set(ticker3))

In [5]:
tickers = ticker1 + ticker2 + ticker3

tickers = list(set(tickers))
len(tickers)

8934

In [6]:
tickers = [tic.strip() for tic in tickers]
tickers = list(set(tickers))
len(tickers)

6191

## 0801 tickers

In [34]:
temp = pd.read_csv('./20230801.csv')

tickers = []
tickers.extend(temp['tck_iem_cd'].values.flatten().tolist())

rld_tickers = temp['rld_ose_iem_tck_cd'].apply(lambda x:x.split(',') if x!='_' else [])

for i in rld_tickers.values:
    tickers.extend(i)
    
tickers = [tic.strip().lower() for tic in tickers]

tickers = list(set(tickers))

## Crawling

In [36]:
agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
headers = {'user-agent':agent}

In [37]:
company_info = {}
company_industry = {}

parse_err = []
info_err = []
indust_err = []
unavailable_ticker = []

In [38]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

for ticker in tqdm(tickers):
#for ticker in tqdm(info_err):
    url = f'https://stockanalysis.com/stocks/{ticker}/company/'
    
    try:
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        parse_err.append(ticker)
        continue
    try:
        check = soup.select("main > div > div.mb-4.text-2xl.font-bold.sm\:text-3xl")[0].text
        if '404' in check:
            unavailable_ticker.append(ticker)
            continue
    except:
        pass
    
    try:
        desc = soup.select("main > div:nth-child(3) > div:nth-child(1)")
        if len(desc)==0:
            desc = soup.select("main > div.mt-4.sm\:mt-5 > div:nth-child(1)")
        company_info[ticker] = desc[0].text
    except:
        info_err.append(ticker)
        
    try:
        num = 0
        selector = 1
        for i in range(1,7):
            industry = soup.select(f'main > div:nth-child(3) > div.lg\:float-right.lg\:w-\[336px\] > div.mt-7.rounded.border.border-gray-200.bg-gray-50.px-3.pt-3.pb-2.dark\:border-dark-700.dark\:bg-dark-775.xs\:px-4.xs\:pt-4.lg\:mt-1 > table > tbody > tr:nth-child({i}) > td.py-1\.5.px-1.font-semibold.lg\:py-2')
            if len(industry)==0:
                selector = 2
                industry = soup.select(f'main > div.mt-4.sm\:mt-5 > div.lg\:float-right.lg\:w-\[336px\] > div.mt-7.rounded.border.border-gray-200.bg-gray-50.px-3.pt-3.pb-2.dark\:border-dark-700.dark\:bg-dark-775.xs\:px-4.xs\:pt-4.lg\:mt-1 > table > tbody > tr:nth-child({i}) > td.py-1\.5.px-1.font-semibold.lg\:py-2')
            if industry[0].text=='Industry':
                num = i
                break
        if num==0:
            indust_err.append(ticker)
        else:
            if selector == 1:
                text = soup.select(f"main > div:nth-child(3) > div.lg\:float-right.lg\:w-\[336px\] > div.mt-7.rounded.border.border-gray-200.bg-gray-50.px-3.pt-3.pb-2.dark\:border-dark-700.dark\:bg-dark-775.xs\:px-4.xs\:pt-4.lg\:mt-1 > table > tbody > tr:nth-child({num}) > td.py-1\.5.px-1.text-right.lg\:py-2 > a")
            else:
                text = soup.select(f"main > div.mt-4.sm\:mt-5 > div.lg\:float-right.lg\:w-\[336px\] > div.mt-7.rounded.border.border-gray-200.bg-gray-50.px-3.pt-3.pb-2.dark\:border-dark-700.dark\:bg-dark-775.xs\:px-4.xs\:pt-4.lg\:mt-1 > table > tbody > tr:nth-child({num}) > td.py-1\.5.px-1.text-right.lg\:py-2 > a")
                
            company_industry[ticker] = text[0].text
    except:
        indust_err.append(ticker)
        
    time.sleep(2)
    if len(info_err)>1:
        print('error')
        break
        
info_err = list(set(info_err)-set(list(company_info.keys()))-set(unavailable_ticker))
indust_err = list(set(indust_err)-set(list(company_industry.keys()))-set(unavailable_ticker))
unavailable_ticker = list(set(unavailable_ticker)-set(list(company_info.keys()))-set(list(company_industry.keys())))
parse_err = list(set(parse_err)-set(unavailable_ticker)-set(list(company_info.keys()))-set(list(company_industry.keys())))

print('파씽 실패 :',len(parse_err))
print('존재하지 않는 티커코드 :',len(unavailable_ticker))
print('기업 설명 실패 :',len(info_err))
print('사업분야 실패 :',len(indust_err))

print('기업 설명 :',len(company_info))
print('기업 사업분야 :',len(company_industry))

100%|███████████████████████████████████████| 1428/1428 [54:33<00:00,  2.29s/it]

파씽 실패 : 0
존재하지 않는 티커코드 : 137
기업 설명 실패 : 0
사업분야 실패 : 0
기업 설명 : 1291
기업 사업분야 : 1291


In [43]:
desc = 'main > div.mt-6.lg\:grid.lg\:grid-cols-sidebar_wide.lg\:gap-x-10 > div.space-y-6.lg\:order-2.lg\:pt-1 > div.px-0\.5.lg\:px-0 > p'
holdings = 'main > div.mt-6.lg\:grid.lg\:grid-cols-sidebar_wide.lg\:gap-x-10 > div.space-y-6.lg\:order-2.lg\:pt-1 > div:nth-child(4) > table'

In [147]:
#etf_info = {}
#etf_holdings = {}

for ticker in tqdm(unavailable_ticker[136:]):
    url = f'https://stockanalysis.com/etf/{ticker}'

    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    try:
        check = soup.select("main > div > div.mb-4.text-2xl.font-bold.sm\:text-3xl")[0].text
        if '404' in check:
            #unavailable_ticker.append(ticker)
            print(ticker,'pass')
            continue
    except:
        pass
    
    about = soup.select(desc)
    table = soup.find_all('table')
    print(ticker,len(about),len(table))
    dfs = pd.read_html(str(table))
    
    holding_list = None
    for df in dfs:
        if 'Symbol'in df.columns:
            holding_list = df
            break
    
    etf_info[ticker] = about[0].text
    etf_holdings[ticker] = holding_list
    
    time.sleep(2)

  0%|                                                     | 0/1 [00:00<?, ?it/s]

ppa 1 4


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.92s/it]


In [149]:
unavailable_ticker = list(set(unavailable_ticker)-set(list(etf_info.keys()))-set(list(etf_holdings.keys())))
len(unavailable_ticker)

35

In [150]:
len(etf_info),len(etf_holdings)

(102, 102)

In [41]:
import pickle

with open('./company_describe.pkl','wb') as f:
    pickle.dump(company_info,f)

with open('./company_industry.pkl','wb') as f:
    pickle.dump(company_industry,f)

In [156]:
with open('./etf_describe.pkl','wb') as f:
    pickle.dump(etf_info,f)

with open('./etf_holdings.pkl','wb') as f:
    pickle.dump(etf_holdings,f)